# install librarys

In [1]:
! pip install transformers
! pip install tensorflow_addons
#! pip install git+https://www.github.com/keras-team/keras-contrib.git


     |████████████████████████████████| 1.9MB 5.3MB/s 
     |████████████████████████████████| 3.2MB 17.9MB/s 
     |████████████████████████████████| 890kB 35.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=11b3623cf014737243e24ca4ee39787a3dc08d9babc492531f6bbbdb82082806
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 706kB 3.8MB/s 


In [2]:
transformers.__version__

NameError: ignored

In [89]:
tf.__version__

'2.4.1'

In [90]:
pd.__version__

'1.1.5'

In [91]:
np.__version__

'1.19.5'

In [3]:
! pip install focal_loss 
import focal_loss 

In [93]:
focal_loss.__version__

'0.0.6'

In [4]:
from focal_loss import sparse_categorical_focal_loss,SparseCategoricalFocalLoss

In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import transformers
from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from sklearn.preprocessing import MultiLabelBinarizer
from itertools import repeat
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from transformers import TFBertModel,  BertConfig, BertTokenizerFast
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy,BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy,BinaryAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split

#from keras_contrib.layers import CRF

# Preprocessing text

In [6]:

labels_dict = {
'O'     : 0,    
'B-art' :   1,  
'B-eve' :   2,
'B-geo' : 3,
'B-gpe' : 4,
'B-nat' :   5,
'B-org' : 6,
'B-per' : 7,
'B-tim' : 8,
'I-art' :   9,
'I-eve' :   10,
'I-geo' :  11,
'I-gpe' :   12,
'I-nat' :    13,
'I-org' :  14,
'I-per' : 15,
'I-tim' :  16
}


In [7]:
data = pd .read_csv('/content/drive/MyDrive/NER Dataset/processed_ner_data_5.csv')

ner =  pd.DataFrame({
    'sentences' : data['sentences'],
    'tag_str' : data['tag_str']
})

ner_tag_list = list(map(lambda x: x.split(',')[1:], ner['tag_str'] ))
ner['ner_tag_list'] = ner_tag_list

ner_tag_list_max_len =list(map(lambda x :  ['O']+x + ( ['O']*(127-len(x))),ner['ner_tag_list']))
ner['ner_tag_list_max_len_with_cls'] = ner_tag_list_max_len

def tag_to_var(x):
  return list(map(lambda k : labels_dict[k],x))

ner_tag_list_max_len_with_cls_to_var = list(map(lambda x : tag_to_var(x), ner['ner_tag_list_max_len_with_cls'] ))
ner['ner_tag_list_max_len_with_cls_to_var'] = ner_tag_list_max_len_with_cls_to_var

ner.head()

,sentences,tag_str,ner_tag_list,ner_tag_list_max_len_with_cls,ner_tag_list_max_len_with_cls_to_var
0,Thousands of demonstrators have marched throu...,",O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[O, O, O, O, O, O, O, B-geo, O, O, O, O, O, B-...","[0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, ..."
1,Families of soldiers killed in the conflict j...,",O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-per,O,O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,They marched from the Houses of Parliament to...,",O,O,O,O,O,O,O,O,O,O,O,B-geo,I-geo,O","[O, O, O, O, O, O, O, O, O, O, O, B-geo, I-geo...","[O, O, O, O, O, O, O, O, O, O, O, O, B-geo, I-...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 11, 0,..."
3,"Police put the number of marchers at 10,000 w...",",O,O,O,O,O,O,O,O,O,O,O,O,O,O,O","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,The protest comes on the eve of the annual co...,",O,O,O,O,O,O,O,O,O,O,O,B-geo,O,O,B-org,I-org,O...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, O, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, B-geo, O,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, ..."


# Modelling

In [8]:
from transformers import AutoTokenizer, AutoModel,TFAutoModel
tokenizer = AutoTokenizer.from_pretrained("ugaray96/biobert_ncbi_disease_ner")
transformer_model = TFAutoModel.from_pretrained("ugaray96/biobert_ncbi_disease_ner")
max_length = 128

Some layers from the model checkpoint at ugaray96/biobert_ncbi_disease_ner were not used when initializing TFBertModel: ['dropout_37', 'classifier']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at ugaray96/biobert_ncbi_disease_ner and are newly initialized: ['bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
train = ner.copy()

In [10]:
# Ready output data for the model
y = np.array(list(train['ner_tag_list_max_len_with_cls_to_var']),dtype=float)
# Tokenize the input (takes some time)
x = tokenizer(
    text=train['sentences'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [11]:
# Load the MainLayer
bert = transformer_model.layers[0]
# Build your model input
input_ids = Input(shape=(max_length,), dtype='int32')
token_type_ids = Input(shape=(max_length,), dtype='int32')
attention_mask = Input(shape=(max_length,), dtype='int32')


inputs = {'input_ids': input_ids,'token_type_ids':token_type_ids,'attention_mask':attention_mask}
# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[0]
#dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = Dropout(0.1)(bert_model, training=False)


#pooled_output = Bidirectional(LSTM(units = 50, return_sequences=True, recurrent_dropout=0.1))(pooled_output)
pooled_output = Bidirectional(LSTM(units = 50, return_sequences=True))(pooled_output)

pooled_output = TimeDistributed(Dense(17))(pooled_output)

# And combine it all in a model object
model = Model(inputs=inputs, outputs=pooled_output, name='BERT_ner')
# Take a look at the model
model.summary()

Model: "BERT_ner"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_3[0][0]                    
                                                                 input_1[0][0]             

In [12]:
#model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])  

#loss = SigmoidFocalCrossEntropy(alpha=0.15, gamma=5.0)
loss = SparseCategoricalFocalLoss(gamma=5.0)
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])  


In [ ]:
history=model.fit(x={'input_ids': x['input_ids'],'token_type_ids':x['token_type_ids'],'attention_mask':x['attention_mask']},y=y,
                  #validation_data=({'input_ids': x_test['input_ids'],'token_type_ids':x_test['token_type_ids'],'attention_mask':x_test['attention_mask']},y_test),
                  batch_size=4,epochs=1)

 2270/11990 [====>.........................] - ETA: 44:27 - loss: -16163.3567 - accuracy: 0.5080

In [40]:
a = model.predict({'input_ids': x['input_ids'][0],'token_type_ids':x['token_type_ids'][0],'attention_mask':x['attention_mask'][0]})
a = np.transpose(a,(1,0,2))
a = a[0]
list(map(lambda x : list(x).index(max(list(x))), a ))

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
list(map(lambda x : list(x).index(max(list(x))), a ))

In [13]:
len(x['input_ids'])

47959

In [41]:
pred_train = []
for i in range(0,1000):
  a = model.predict({'input_ids': x['input_ids'][i],'token_type_ids':x['token_type_ids'][i],'attention_mask':x['attention_mask'][i]})
  a = np.transpose(a,(1,0,2))
  a = a[0]
  pred_train.append(list(map(lambda x : list(x).index(max(list(x))), a )))

print(classification_report( y[0:1000].ravel()   ,  pred_train.ravel()    )) 

In [47]:
pred_train = np.array(pred_train)

In [65]:
y[0:1000].ravel()

array([0., 0., 0., ..., 0., 0., 0.])

In [58]:
pred_train.shape

(1000, 128)

In [67]:
print(classification_report( y[0:1000].ravel()   ,  pred_train.ravel()    )) 

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99    124767
         1.0       0.00      0.00      0.00        39
         2.0       0.00      0.00      0.00        20
         3.0       0.00      0.00      0.00       583
         4.0       0.00      0.00      0.00       543
         5.0       0.00      0.00      0.00         9
         6.0       0.00      0.00      0.00       413
         7.0       0.00      0.00      0.00       327
         8.0       0.00      0.00      0.00       358
         9.0       0.00      0.00      0.00        22
        10.0       0.00      0.00      0.00        16
        11.0       0.00      0.00      0.00       105
        12.0       0.00      0.00      0.00        26
        13.0       0.00      0.00      0.00         5
        14.0       0.00      0.00      0.00       290
        15.0       0.00      0.00      0.00       400
        16.0       0.00      0.00      0.00        77

    accuracy              

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [115]:
a=[]
a.append([1,2,3])
a.append([1,2,3])
a

[[1, 2, 3], [1, 2, 3]]

In [116]:
['a']*-3

[]